# Static and dynamic gestures (Mac OS X)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
from skimage import io
import numpy as np
import cv2

from thread_camera import WebcamVideoStream
from utils import load_image_with_alpha, overlay_alpha
from utils import draw_text, draw_multiline_text

from jesture_sdk_python import JestureSdkRunner

print('cv2.__version__:', cv2.__version__)  # 4.1.2 recommended

shutil.which("libSystem.B.dylib"): None
ctypes.CDLL("libSystem.B.dylib")._name: libSystem.B.dylib
ctypes.__version__: 1.1.0
platform.mac_ver(): ('10.16', ('', '', ''), 'x86_64')


(MainThread) Loaded backend module://ipykernel.pylab.backend_inline version unknown.


cv2.__version__: 4.1.2


In [3]:
# create the application window
name = 'JestureSDK: Python Demo'
width, height = (640, 480)
cv2.namedWindow(name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(name, (width, height))
cv2.startWindowThread()

# load the logo image
design_root = '/Users/izakharkin/Desktop/deepjest/_design'
logo_path = f'{design_root}/wix/jesture_ai_logo_comfortaa/jesture_logo_comfortaa-removebg.png'
logo_img, logo_alpha = load_image_with_alpha(logo_path, remove_borders=True)

# set the ui elements positions
left_box_tl = (70, 360)
left_box_br = (200, 420)

right_box_tl = (450, 360)
right_box_br = (580, 420)

# set the text positions
logo_loc = (10, 10)
left_text_loc = (80, 390)
right_text_loc = (460, 390)
dynamic_text_loc = ((left_text_loc[0] + right_text_loc[0]) // 2, 50)
font = ImageFont.truetype("Comfortaa-Light.ttf", 24)

(MainThread) STREAM b'IHDR' 16 13
(MainThread) STREAM b'IDAT' 41 8192


Removing 250 pixels from up and down borders
Original image size: (1920, 580)
Target size: (192, 58)


In [4]:
# start Jesture SDK Python runner
jesture_runner = JestureSdkRunner(cam_id=1)
jesture_runner.start_recognition()

(MainThread) [JestureSdkRunner] Instance created.
(jesture_sdk_python_thread) [JestureSdkRunner] Starting recognition...
(MainThread) [JestureSdkRunner] Recognition thread started.


In [5]:
# jesture_runner.stop_recognition()

In [6]:
# start reading frames to display in the application window
cap = ThreadCamera(cam_id=1, width=width, height=height)
cap.start()

In [7]:
selfie_mode = True
i = 0
while(True):
    if cap.frame is None:
        continue
    
    # get current webcam image
    frame = cap.frame[:,::-1,:] if selfie_mode else cap.frame  # TODO: read frames from dylib
    
    # get current hand gestures
    dynamic_gesture = jesture_runner.get_gesture('dynamic')
    left_gesture = jesture_runner.get_gesture('left_static')
    right_gesture = jesture_runner.get_gesture('right_static')
    
    # draw logo
    frame = overlay_alpha(logo_img[:,:,::-1], logo_alpha, frame, loc=logo_loc, alpha=1.0)
    
    # draw ui elements
    frame = Image.fromarray(frame)
    draw = ImageDraw.Draw(frame, "RGBA")
    draw.rectangle((left_box_tl, left_box_br), fill=(0, 0, 0, 127), outline=(235, 190, 63, 127))
    draw.rectangle((right_box_tl, right_box_br), fill=(0, 0, 0, 127), outline=(235, 190, 63, 127))
    
    # ellipse with emoji images instead of rectangle 
#     leftUpPoint = (x-r, y-r)
#     rightDownPoint = (x+r, y+r)
#     twoPointList = [leftUpPoint, rightDownPoint]
#     draw.ellipse(twoPointList, fill=(255,0,0,255))
#     draw.ellipse((100, 100, 150, 200), fill=(0, 0, 0, 127), outline=(235, 190, 63, 127))
#     draw.ellipse((100, 100, 150, 200), fill=(0, 0, 0, 127), outline=(235, 190, 63, 127))
#     frame = np.array(frame).astype(np.uint8)
    
    # draw text
    draw.text(dynamic_text_loc, dynamic_gesture, font=font)
    draw.text(left_text_loc, left_gesture, font=font)
    draw.text(right_text_loc, right_gesture, font=font)
    frame = np.array(frame).astype(np.uint8)

    cv2.imshow(name, frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    i += 1
    
jesture_runner.stop_recognition()
cap.stop()

cv2.waitKey(1)
cv2.destroyWindow(name)
cv2.destroyAllWindows()
cv2.waitKey(1)

(MainThread) [JestureSdkRunner] Stopping recognition...
(MainThread) [JestureSdkRunner] Recognition stopped.
(MainThread) [JestureSdkRunner] Thread joined.
(MainThread) [WebcamVideoStream] Stopping...
(thread_camera_python) [WebcamVideoStream] Frame loop finished.
(thread_camera_python) [WebcamVideoStream] Capture released.
(MainThread) [WebcamVideoStream] Camera thread joined.


-1

In [8]:
# jesture_runner.stop_recognition()
# cap.stop()

# cv2.waitKey(1)
# cv2.destroyWindow(name)
# cv2.destroyAllWindows()
# cv2.waitKey(1)

---

In [ ]:
# https://unicode.org/emoji/charts/emoji-list.html#1f44c
gesture_to_emoji = {
    '': '',
    '———': '———',
    'ONE': '\U0000261D',
    'TWO': 'TWO',
    'THREE': 'THREE',
    'FOUR': 'FOUR',
    'FIVE' : '\U0001F590',
    'OK': '\U0001F44C',
    'YEAH': '\U0000270C',
    'SPIDERMAN': '\U0001F91F',
    'ROCK': '\U0001F918',
    'FIST': '\U00009270A'
}

# !pip install emoji
import unicodedata
# in python2 use u'\U0001f603'
print('\U0001F44C')#U+1F44C
print(gesture_to_emoji['FIVE'])

In [ ]:
def show_logo_alpha():
    design_root = '/Users/izakharkin/Desktop/deepjest/_design'
    logo_path = f'{design_root}/wix/jesture_ai_logo_comfortaa/jesture_logo_comfortaa-removebg.png'

    logo_img, logo_alpha = load_image_with_alpha(logo_path, remove_borders=True)

    plt.imshow(logo_img);
    plt.show();
    plt.imshow(logo_alpha, cmap='gray');
    plt.show();

In [ ]:
def test_overlay():
    testimg = io.imread('./test.jpg')
    testimg = overlay_alpha(logo_img, logo_alpha, testimg, loc=(10, 10), alpha=1.0)

    idx_to_gesture = {1: 'one', 2: 'two', 3: 'three', 4: 'four', 5: 'five',
                      6: 'fist', 7: 'piece', 8: 'love', 9: 'ok', 0: 'horns'}
    help_textlist = [f'{k}: {v}' for k, v in idx_to_gesture.items()]
    testimg = draw_multiline_text(testimg, help_textlist)

    plt.figure(figsize=(10, 7))
    plt.imshow(testimg);

In [ ]:
def test_blackbox():
    blackbox = io.imread('./blackbox.png')
    orig_size = (blackbox.shape[1], blackbox.shape[0])
    print('orig_size:', orig_size)
    target_size = (orig_size[0] // 7, orig_size[1] // 7)
    print('target_size:', target_size)
    blackbox = cv2.resize(blackbox, target_size)
    plt.imshow(blackbox);
    plt.show();

    blackbox_alpha = np.ones_like(blackbox)[:,:,0] * 255
    blackbox_alpha[blackbox[:,:,0] == 255] = 0
    plt.imshow(blackbox_alpha, cmap='gray');
    plt.show();

    testimg = overlay_alpha(blackbox, blackbox_alpha, testimg, loc=(10, 100), alpha=1.0)
    plt.figure(figsize=(10, 7))
    plt.imshow(testimg);

In [ ]:
def test_pil_draw():
    testimg = io.imread('./test.jpg')
    testimg = Image.fromarray(testimg)
    draw = ImageDraw.Draw(testimg, "RGBA")
    draw.rectangle(((280, 10), (1010, 706)), fill=(0, 0, 0, 127))
    draw.rectangle(((280, 10), (1010, 706)), outline=(63, 190, 235, 127))
    plt.figure(figsize=(12, 10))
    plt.imshow(np.array(testimg));